# 01 - Ingest and Index Documents

This notebook handles document chunking, embedding generation, and FAISS index creation.

## Objectives
- Load and chunk documents
- Generate embeddings for all chunks
- Build FAISS index for similarity search
- Create docstore mapping for chunk retrieval

In [ ]:
# Import required libraries
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import logging

# Add src to path
sys.path.append('../src')

# Import raglab modules
from core.io import DataLoader
from chunking import chunk_text
from indexing.index import EmbeddingProvider, LocalFAISSIndex, RAGRetriever
from core.interfaces import Chunk

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports successful")

## Configuration

Set up chunking and embedding parameters.

In [ ]:
# Configuration parameters
CHUNK_SIZE = 500  # Characters per chunk
CHUNK_OVERLAP = 100  # Character overlap between chunks
EMBEDDING_DIM = 768  # Dimension of embeddings (adjust based on your model)

# Your embedding function from setup notebook
def your_embedding_function(texts: list) -> np.ndarray:
    """
    Replace this with your actual embedding function.
    This should return embeddings with shape (len(texts), EMBEDDING_DIM)
    """
    # Mock implementation - replace with real embeddings
    return np.random.random((len(texts), EMBEDDING_DIM))

print(f"📋 Configuration:")
print(f"   Chunk size: {CHUNK_SIZE} characters")
print(f"   Chunk overlap: {CHUNK_OVERLAP} characters")
print(f"   Embedding dimension: {EMBEDDING_DIM}")
print("⚠️  Remember to replace the mock embedding function with a real one")

## Load and Chunk Documents

In [ ]:
# Load corpus
loader = DataLoader(base_path='..')
corpus_df = loader.load_corpus('data/corpus.parquet')

print(f"📚 Loaded corpus with {len(corpus_df)} documents")
print("\n📖 Sample documents:")
for idx, row in corpus_df.head(3).iterrows():
    print(f"  {row['doc_id']}: {row['text'][:80]}...")

In [ ]:
# Chunk all documents
all_chunks = []

for idx, row in corpus_df.iterrows():
    doc_id = row['doc_id']
    text = row['text']
    
    # Chunk the document
    chunks = chunk_text(
        text=text,
        document_id=doc_id,
        chunk_size=CHUNK_SIZE,
        overlap=CHUNK_OVERLAP
    )
    
    all_chunks.extend(chunks)
    print(f"📄 {doc_id}: {len(chunks)} chunks")

print(f"\n✅ Total chunks created: {len(all_chunks)}")

# Display sample chunks
print("\n📊 Sample chunks:")
for i, chunk in enumerate(all_chunks[:3]):
    print(f"  {chunk.chunk_id}: {chunk.text[:60]}...")
    print(f"    Document: {chunk.document_id}, Length: {len(chunk.text)} chars")

## Generate Embeddings

In [ ]:
# Create embedding provider
embedding_provider = EmbeddingProvider(your_embedding_function)

# Extract chunk texts for embedding
chunk_texts = [chunk.text for chunk in all_chunks]
chunk_ids = [chunk.chunk_id for chunk in all_chunks]

print(f"🔄 Generating embeddings for {len(chunk_texts)} chunks...")
print("   This may take a moment depending on your embedding provider")

# Generate embeddings
embeddings = embedding_provider.embed_texts(chunk_texts)

print(f"✅ Generated embeddings with shape: {embeddings.shape}")

# Save embeddings
loader.save_embeddings(embeddings, 'artifacts/embeddings.npy')
print("💾 Saved embeddings to artifacts/embeddings.npy")

## Build FAISS Index

In [ ]:
# Create FAISS index
faiss_index = LocalFAISSIndex(
    embedding_dim=EMBEDDING_DIM,
    index_type="flat"  # Use "ivf" or "hnsw" for larger datasets
)

# Add embeddings to index
faiss_index.add_embeddings(embeddings, chunk_ids)

print(f"✅ Built FAISS index with {len(chunk_ids)} vectors")

# Save index
faiss_index.save('artifacts/faiss.index')
print("💾 Saved FAISS index to artifacts/faiss.index")

## Create Docstore

In [ ]:
# Create docstore mapping chunk_id -> text/metadata
docstore_data = []

for chunk in all_chunks:
    docstore_data.append({
        'chunk_id': chunk.chunk_id,
        'text': chunk.text,
        'document_id': chunk.document_id,
        'metadata': chunk.metadata or {}
    })

docstore_df = pd.DataFrame(docstore_data)

# Save docstore
loader.save_docstore(docstore_df, 'artifacts/docstore.parquet')

print(f"✅ Created docstore with {len(docstore_df)} entries")
print("💾 Saved docstore to artifacts/docstore.parquet")

# Display sample docstore entries
print("\n📊 Sample docstore entries:")
print(docstore_df.head())

## Test Retrieval

In [ ]:
# Create retriever and test it
retriever = RAGRetriever(
    embedding_provider=embedding_provider,
    docstore_path='../artifacts/docstore.parquet',
    index_path='../artifacts/faiss.index'
)

# Test query
test_query = "What is a copayment?"
results = retriever.retrieve(test_query, k=3)

print(f"🔍 Test query: '{test_query}'")
print(f"📊 Retrieved {len(results)} results:")

for i, result in enumerate(results):
    print(f"\n  {i+1}. Chunk {result.chunk_id} (score: {result.similarity_score:.3f})")
    print(f"     Text: {result.chunk_text[:100]}...")

print("\n✅ Retrieval test successful!")

## Summary

Index creation completed successfully!

In [ ]:
# Summary statistics
print("📈 Indexing Summary:")
print(f"   Documents processed: {len(corpus_df)}")
print(f"   Total chunks created: {len(all_chunks)}")
print(f"   Embedding dimension: {embeddings.shape[1]}")
print(f"   FAISS index size: {len(chunk_ids)} vectors")

print("\n📁 Generated artifacts:")
artifacts_dir = Path('../artifacts')
if artifacts_dir.exists():
    for file in artifacts_dir.iterdir():
        if file.is_file():
            size_mb = file.stat().st_size / (1024 * 1024)
            print(f"   {file.name}: {size_mb:.2f} MB")

print("\n🎉 Ready for notebook 02_retrieval_eval.ipynb!")